In [ ]:
import numpy as np
import pandas as pd 


## Machine Learning and Data Processing Worksheet 
Once you have gathered your datasets you can import them into this work sheet. This work sheet will help you load a dataset, look at your data and train models on it. As there are some quite challenging concepts, some of the code is written for you. Fill in where you can, and try understand what is happening. Can you increase model accuracy? 

### Load in the data

### Visualise the data

### Machine Learning